In [1]:

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import glob
import h5py

def ls2sol(ls):
    N_s = 668.6
    ls_peri = 250.99
    t_peri = 485.35
    a = 1.52368
    e = 0.09340
    epsilon = 25.1919
    if (ls == 0).any():
        ls = .01
    nu = np.radians(ls) + 1.90258
    E = np.arctan((np.tan(nu/2))/(np.sqrt((1+e)/(1-e))))*2
    M = E - e*np.sin(E)
    Ds = (M/(2*np.pi))*N_s + t_peri
    if (Ds < 0).any():
        Ds = Ds + N_s
    if (Ds > N_s).any():
        Ds = Ds - N_s
    return(Ds)

sg = xr.open_dataset('StupidGridFull.nc', group='flux')
files = ['redsun_timeseries_17_00.nc', 'redsun_timeseries_17_10.nc']

j2AAF = np.zeros((19,37))
j2AAi_bg1_F = np.zeros((19,37))
j2AAi_bg2_F = np.zeros((19,37))

for file in files:
    ds = xr.open_dataset(file)
    lat = ds['lat'][0]
    lon = ds['lon'][0]
    G = np.zeros(len(ds['lon']))
    for ri in range(0,len(ds['lon'])):
        ls = ds['ls'][ri]
        hr = ds['hr'][ri]
        G[ri] = sg['flux_dw_sw'][lat,lon,ls,hr]
    lss = np.unique(ds['ls'])

    Z = 2.016
    n = 2
    F = 96485.33212
    P = G[:, np.newaxis, np.newaxis] * ds['j2_etaPEC_AA_2bg'] * Z/(n*F)
    zz = np.zeros((len(lss),len(ds['j2-bg1']),len(ds['j2-bg2'])))
    for i in range(0,len(lss)):
        hr_int = np.where(ds['ls']==lss[i])
        x = np.array(ds['hr'][hr_int] * 2 * 1.02569 * 60 * 60)
        for j in range(0,len(ds['j2-bg1'])):
            for k in range(0,len(ds['j2-bg2'])):
                y = P[:,j,k][hr_int]
                z = np.trapz(y,x=x)
                zz[i,j,k] = z
    z = np.zeros((len(ds['j2-bg1']),len(ds['j2-bg2'])))
    x = ls2sol(lss) * 1.02569 * 24 * 60 * 60
    for j in range(0,len(ds['j2-bg1'])):
        for k in range(0,len(ds['j2-bg2'])):
            y = zz[:,j,k]
            z[j,k] = np.trapz(y,x=lss)
    j2AA = np.max(z)
    j2AAi = np.unravel_index(np.argmax(z),np.shape(z), order='C')

    j2AAF[lat,lon] = j2AA * (1/688)
    j2AAi_bg1_F[lat,lon] = ds['j2-bg1'][j2AAi[0]]
    j2AAi_bg2_F[lat,lon] = ds['j2-bg1'][j2AAi[1]]

lats = np.arange(-90,91,10)
lons = np.arange(-180,181,10)
test = xr.Dataset(
    {'j2AA': (('lon','lat'), j2AAF.transpose()),
    'j2AA-bg1': (('lon','lat'), j2AAi_bg1_F.transpose()),
    'j2AA-bg2': (('lon','lat'), j2AAi_bg2_F.transpose())},
    coords = {
        'lon': lons,
        'lat': lats,
    },
)
test['lat'].attrs['units'] = 'degrees_north'
test['lon'].attrs['units'] = 'degrees_east'
test['j2AA'].attrs['units'] = '(g)/(day*m^2)'
test['j2AA-bg1'].attrs['units'] = 'eV'
test['j2AA-bg2'].attrs['units'] = 'eV'

test['j2AA'] = test['j2AA'].where(test['j2AA'] != 0.0)
test['j2AA-bg1'] = test['j2AA-bg1'].where(test['j2AA-bg1'] != 0.0)
test['j2AA-bg2'] = test['j2AA-bg2'].where(test['j2AA-bg2'] != 0.0)


test.to_netcdf('j2AA.nc')

In [2]:
test

<xarray.Dataset>
Dimensions:   (lat: 19, lon: 37)
Coordinates:
  * lon       (lon) int64 -180 -170 -160 -150 -140 -130 ... 140 150 160 170 180
  * lat       (lat) int64 -90 -80 -70 -60 -50 -40 -30 ... 30 40 50 60 70 80 90
Data variables:
    j2AA      (lon, lat) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    j2AA-bg1  (lon, lat) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    j2AA-bg2  (lon, lat) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [4]:
ds =  xr.open_dataset('StupidGridFullDustStorm.nc')

/Users/aaronberliner/anaconda3/lib/python3.6/site-packages/xarray/coding/times.py:419: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  self.use_cftime)
/Users/aaronberliner/anaconda3/lib/python3.6/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


In [18]:
ds['aerolongitudes']

<xarray.DataArray 'aerolongitudes' (ls: 99)>
array([180.27612, 182.02138, 183.77563, 185.53888, 187.31104, 189.09206,
       190.88185, 192.68034, 194.48743, 196.303  , 198.12692, 199.95908,
       201.79929, 203.64743, 205.5033 , 207.3667 , 209.23746, 211.11536,
       213.00015, 214.8916 , 216.78947, 218.6935 , 220.6034 , 222.51886,
       224.43962, 226.36536, 228.29575, 230.23047, 232.16917, 234.11153,
       236.05716, 238.00572, 239.95683, 241.91011, 243.8652 , 245.82172,
       247.77925, 249.73744, 251.69586, 253.65416, 255.61192, 257.5688 ,
       259.52432, 261.47815, 263.42993, 265.37927, 267.3258 , 269.26913,
       271.20892, 273.14484, 275.0765 , 277.00363, 278.92587, 280.8429 ,
       282.75443, 284.6602 , 286.55984, 288.4532 , 290.33994, 292.21985,
       294.0927 , 295.95825, 297.81635, 299.66672, 301.50925, 303.34375,
       305.17007, 306.98807, 308.79764, 310.5986 , 312.39093, 314.1745 ,
       315.94922, 317.71506, 319.47192, 321.2198 , 322.95862, 324.6884 ,
      

In [34]:
a = np.floor(np.linspace(0,98,13))

In [39]:
ds

<xarray.Dataset>
Dimensions:         (lat: 19, level: 20, lon: 37, ls: 99, time: 13, wl: 1341)
Coordinates:
  * time            (time) object 0001-01-01 00:00:00 ... 0001-01-02 00:00:00
Dimensions without coordinates: lat, level, lon, ls, wl
Data variables:
    altitude        (level) float32 ...
    latitude        (lat) float32 ...
    longitude       (lon) float32 ...
    aerolongitudes  (ls) float32 180.27612 182.02138 ... 357.4227 358.97162
    wavelength      (wl) float32 ...
    albedo          (lat, lon) float32 ...

In [23]:
ds

<xarray.Dataset>
Dimensions:         (lat: 19, level: 20, lon: 37, ls: 99, time: 13, wl: 1341)
Coordinates:
  * time            (time) object 0001-01-01 00:00:00 ... 0001-01-02 00:00:00
Dimensions without coordinates: lat, level, lon, ls, wl
Data variables:
    altitude        (level) float32 ...
    latitude        (lat) float32 ...
    longitude       (lon) float32 ...
    aerolongitudes  (ls) float32 180.27612 182.02138 ... 357.4227 358.97162
    wavelength      (wl) float32 ...
    albedo          (lat, lon) float32 ...

In [25]:
dict(ls=np.round(np.linspace(0,99,13)))

{'ls': array([ 0.,  8., 16., 25., 33., 41., 50., 58., 66., 74., 82., 91., 99.])}

In [26]:
ds['ls']

<xarray.DataArray 'ls' (ls: 99)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95, 96, 97, 98])
Dimensions without coordinates: ls

In [58]:
a = np.floor(np.linspace(0,98,13)).astype(int)
b = ds.isel(ls=a)

In [61]:
b.to_netcdf('StupidGridDustCor.nc')